In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Calculate initial about of gas in the bubbles of given volume
def get_mol_from_r(r,pres = 7.9,t=-1):

    #A Pressure atm
    Temp = t + 273.15 # Temprature °К
    Rgas = 8.20575e-5  # R gas constant m3 atm K−1 mol−1 
    r = r/1000 #  mm to m
    vol = (4/3*np.pi*r**3) # Volume of bubble м^3
    
    gas_mol=1000*(pres*vol)/(Rgas*Temp) #[atm]*[m3]/([ m3 atm k-1 mol-1]*[k])
    #conc = gas_mol / vol
    return gas_mol #/vol 

In [10]:
# Bottom temperature in the chosen station is -1 most times in the year
#Get the initial flux from the bubbles, mmol/m2/sec
b1 = round(get_mol_from_r(4,t=-1)*4,3)
f1 = round(get_mol_from_r(4,t=-1)*3,3)
s1 = round(get_mol_from_r(2,t=-1)*32,3)

print ('Base scenario:',b1)
print ('Small bubbles scenario:',s1)
print ('Reduced Flux:',f1)

Base scenario: 0.379
Small bubbles scenario: 0.379
Reduced Flux: 0.285


Equation (19) From Sander 

$K_H(T) = K_H^\Theta*\exp{(\frac{dlnH^{cp}}{d(1/T)}\cdot(\frac{1}{T}-\frac{1}{T^{\Theta}}))}$

In [3]:
import numpy as np
def get_kh_t(Kh_teta,coef,T):
    kh_t = Kh_teta * np.exp((coef)*(1/T - 1/298.15)) #Mol/m3*Pa
    kh_t_atm = kh_t* 1.e6 #Micromol/l*atm
    return int(kh_t_atm)

In [4]:
Values from the Table in Sander:
Kh_theta = 1.4e-5 # mol/m3 Pa
coef = 1900 #[K]
Kh_theta =  Kh_theta * 101.325 #Convert to M/atm 
print (Kh_theta,'Kh_teta M/atm')
#Correct by temperature, using (19)
T = 0 + 273.15
Kh_t_atm = get_kh_t(Kh_theta,coef,T)*0.863
print (Kh_t_atm,'Kh corrected by Temperat')

0.00141855 Kh_teta M/atm
2192.883 Kh corrected by Temperat


In [27]:
# Correction by salinity 
# Kh(sal) = kh*10**(-Sal[M]*Ksal) 
 
#Ksal value from Gros et al.2018 
#Ksal = 0.127 L/Mole
sal = 35
koef_salt = 10**(-(sal/58)*0.127)
print (koef_salt,'koef_salt')

0.8382278169108212 koef_salt


In [28]:
pressures = range(1,10)
depths = range(0,90,10)

import pandas as pd
df = pd.DataFrame()
df['depths'] = depths
df=df.set_index('depths')
df['kh'] = [int(Kh_theta*1000000) for n in pressures]
df['c_t-1.5_salt'] = [int(get_kh_t(Kh_theta,coef,(-1.5+273.15))*n*koef_salt) for n in pressures]
#df['c_t-1.5_salt'] = df['c_t-1.5']*koef_salt
df['c_t0_salt'] = [int(get_kh_t(Kh_theta,coef,(0+273.15))*n*koef_salt) for n in pressures]
#df['c_t0_salt'] = df['c_t0']*koef_salt
df['c_t1.5_salt'] = [int(get_kh_t(Kh_theta,coef,(1.5+273.15))*n*koef_salt) for n in pressures]
df['c_mean'] = df.mean(axis = 1)


print (df)

          kh  c_t-1.5_salt  c_t0_salt  c_t1.5_salt    c_mean
depths                                                      
0       1418          2213       2129         2051   1952.75
10      1418          4427       4259         4102   3551.50
20      1418          6641       6389         6153   5150.25
30      1418          8855       8519         8204   6749.00
40      1418         11068      10649        10255   8347.50
50      1418         13282      12779        12306   9946.25
60      1418         15496      14909        14358  11545.25
70      1418         17710      17039        16409  13144.00
80      1418         19923      19169        18460  14742.50


Calculate the saturations solubility at different depths